In [1]:
import pyttsx3
import speech_recognition as sr
import webbrowser
import datetime
import pywhatkit
import os
import yfinance as yf
import pyjokes

ERROR_NO_ENTIENDO = "No te entiendo"
ERROR_NO_RESULTADOS = "No te entendi o no puedo encontrar resultados"
INFO_ESTOY_ESPERANDO = "Estoy esperando tus ordenes"


class ObtencionVoz(object):
	"""Modulo para obtener la información de voz"""
	def __init__(self, languaje_habla="spanish",lenguaje_reconocimiento="es-MX", tasa_muestreo=150,volume=1):
		super(ObtencionVoz, self).__init__()
		self.languaje_habla = languaje_habla
		self.tasa_muestreo = tasa_muestreo
		self.volume = volume
		self.lenguaje_reconocimiento = lenguaje_reconocimiento

	def speaking(self,message):
		engine = pyttsx3.init()
		engine.setProperty('rate', self.tasa_muestreo)
		engine.setProperty('voice', self.languaje_habla)
		engine.setProperty('volume', self.volume)
		engine.say(message)
		engine.runAndWait()

	def transform(self):
		r = sr.Recognizer()
		with sr.Microphone() as source:
			r.adjust_for_ambient_noise(source)
			r.pause_threshold = 0.8
			said = r.listen(source)
			try:
				q = r.recognize_google(said, language=self.lenguaje_reconocimiento)
				return q
			except sr.UnknownValueError:
				return ERROR_NO_ENTIENDO
			except sr.RequestError:
				return ERROR_NO_RESULTADOS
			except e:
				return INFO_ESTOY_ESPERANDO





In [24]:
#imagenes 
import cv2
import os
import imutils
import numpy as np
#utilerias
import threading
import time
from datetime import datetime
import hashlib


MUESTRAS_NUMERO = 800

class ReconocimientoFacial(object):
	"""Modulo para hacer el reconocimiento facial y la autenticación"""
	def __init__(self, numero_muestras=MUESTRAS_NUMERO):
		super(ReconocimientoFacial, self).__init__()
		self.voz = ObtencionVoz()
		self.numero_muestras = numero_muestras

	def getHash(self):
		return hashlib.md5(datetime.now().__str__().encode("utf-8")).hexdigest()

	def prepare_working_directory(self,personName):
		""" Crea las carpetas para almacenar las imagenes del directorio."""
		dataPath = os.path.join(os.getcwd(),"data")
		personPath = os.path.join(dataPath,personName)
		if not os.path.exists(personPath):
			print('Carpeta creada: ',personPath)
			os.makedirs(personPath)
		return dataPath,personPath

	def get_samples_faces_from_cam(self,personPath):
		"""Realiza la toma de las capturas de la camara."""
		labels = ["seri@","sonriente","riendo","enojad@", "Guino un ojo","Guino el otro ojo","Con lentes","Sin Lentes","Hablando"]
		periodo = MUESTRAS_NUMERO//len(labels)
		index=0
		cap = cv2.VideoCapture(-1)
		faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
		count = 0
		conta = 0 
		speak = True
		while True:
			ret, frame = cap.read()
			if ret == False: break
			frame =  imutils.resize(frame, width=640)
			gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
			auxFrame = frame.copy()
			faces = faceClassif.detectMultiScale(gray,1.3,5)
			for (x,y,w,h) in faces:
				cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
				rostro = auxFrame[y:y+h,x:x+w]
				rostro = cv2.resize(rostro,(150,150),interpolation=cv2.INTER_CUBIC)
				cv2.imwrite(personPath + '/rotro_{}_{}.jpg'.format(self.getHash(),count),rostro)
				count = count + 1
				conta += 1
			if conta == periodo:
				index= (index+1)%(len(labels))
				conta = 0
				speak = True
			if speak:
				speak = False
				hilo1 = threading.Thread(target=self.voz.speaking,args=("Podrias poner tu cara "+labels[index],))
				hilo1.start()
				
			cv2.putText(frame,labels[index]+" count: "+str(count)+" index: "+str(index), (114,430), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
			cv2.imshow('frame',frame)
			k = cv2.waitKey(1)
			if k == 27 or count >= MUESTRAS_NUMERO:
				break
		cap.release()
		cv2.destroyAllWindows() 

	def train_model(self,working_directory,face_recognizer = cv2.face.LBPHFaceRecognizer_create(),model_xml="modeloLBPHFace.xml"):
		dataPath = working_directory[0]
		personPath = working_directory[1]
		peopleList = [directorio for directorio in os.listdir(dataPath) if not directorio.startswith(".")]
		print('Lista de personas: ', peopleList)
		labels = []
		facesData = []
		label = 0
		for nameDir in peopleList:
			personPath = dataPath + '/' + nameDir
			print('Leyendo las imágenes')
			for fileName in os.listdir(personPath):
				labels.append(label)
				facesData.append(cv2.imread(personPath+'/'+fileName,0))
				#image = cv2.imread(personPath+'/'+fileName,0)
				#cv2.imshow('image',image)
				#cv2.waitKey(10)
			label = label + 1
			
		#face_recognizer = cv2.face.EigenFaceRecognizer_create()
		#face_recognizer = cv2.face.FisherFaceRecognizer_create()
		#face_recognizer = cv2.face.LBPHFaceRecognizer_create()
		print("Entrenando...")
		if(type(face_recognizer) == cv2.face.FisherFaceRecognizer):
			labels[0] = 12
		face_recognizer.train(facesData, np.array(labels))
		face_recognizer.write(model_xml)
		print("Modelo almacenado...")

	def reconocimiento_facial(self,working_directory,face_recognizer = cv2.face.LBPHFaceRecognizer_create(),modelo_almacenado='modeloLBPHFace.xml'):
		dataPath = working_directory[0]
		personPath = working_directory[1]
		imagePaths = os.listdir(dataPath)
		print('imagePaths=',imagePaths)
		#face_recognizer = cv2.face.EigenFaceRecognizer_create()
		#face_recognizer = cv2.face.FisherFaceRecognizer_create()
		#face_recognizer = cv2.face.LBPHFaceRecognizer_create()
		# Leyendo el modelo
		#face_recognizer.read('modeloEigenFace.xml')
		#face_recognizer.read('modeloFisherFace.xml')
		face_recognizer.read(modelo_almacenado)
		cap = cv2.VideoCapture(0)
		#cap = cv2.VideoCapture('Video.mp4')

		faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
		while True:
			ret,frame = cap.read()
			if ret == False: break
			gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
			auxFrame = gray.copy()
			faces = faceClassif.detectMultiScale(gray,1.3,5)
			for (x,y,w,h) in faces:
				rostro = auxFrame[y:y+h,x:x+w]
				rostro = cv2.resize(rostro,(150,150),interpolation= cv2.INTER_CUBIC)
				result = face_recognizer.predict(rostro)
				cv2.putText(frame,'{}'.format(result),(x,y-5),1,1.3,(255,255,0),1,cv2.LINE_AA)
				'''
				# EigenFaces
				if result[1] < 5700:
					cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
					cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
				else:
					cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
					cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
				'''
				'''
				# FisherFace
				if result[1] < 500:
					cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
					cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
				else:
					cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
					cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
				'''
				# LBPHFace
				if result[1] < 70:
					cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
					cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
				else:
					cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
					cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
				
			cv2.imshow('frame',frame)
			k = cv2.waitKey(1)
			if k == 27:
				break
		cap.release()
		cv2.destroyAllWindows()






In [21]:

face_recognition = ReconocimientoFacial()

working_directory = face_recognition.prepare_working_directory("david")
#face_recognition.get_samples_faces_from_cam(working_directory[1])
print("Entrenando modelo.....")
#face_recognition.train_model(working_directory,face_recognizer = cv2.face.LBPHFaceRecognizer_create(),model_xml="modeloLBPHFace.xml")


Entrenando modelo.....


## Fase de evalucion del modelo 

Evaluando los modelos 

Modelos a realizar la evaluación

In [22]:
modelos = [cv2.face.EigenFaceRecognizer_create(),cv2.face.FisherFaceRecognizer_create(),cv2.face.LBPHFaceRecognizer_create(),]
rutas_modelos = ['modeloEigenFace.xml','modeloFisherFace.xml','modeloLBPHFace.xml',]

Entrenado los modelos

In [17]:
from multiprocessing  import Pool
def f(x):
    modelo = x[0]
    ruta = x[1]
    face_recognition = ReconocimientoFacial()
    face_recognition.train_model(working_directory,
                             face_recognizer = modelo
                             ,model_xml=ruta)
    return ruta

with Pool(32) as p:
    result = p.map_async(f, [(m,r) for m,r in zip(modelos,rutas_modelos)])
    result.wait()

In [ ]:
face_recognition.train_model(working_directory,
                         face_recognizer = modelos[1]
                         ,model_xml=rutas_modelos[1])

Carga de data set de entrenamiento

In [26]:
from tqdm import tqdm
def load_data(dir_path, IMG_SIZE):
    faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
    X = []
    yo = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    auxFrame = gray.copy()
                    faces = faceClassif.detectMultiScale(gray,1.3,5)
                    for (x,y,w,h) in faces:
                        rostro = auxFrame[y:y+h,x:x+w]
                        rostro = cv2.resize(rostro,(150,150),interpolation= cv2.INTER_CUBIC)
                        X.append(rostro)
                        yo.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return X, y, labels

In [30]:
PATH = "/home/alan/Documents/source_code/python/data"
TRAIN_DIR = (f'{PATH}/.Training/')
TEST_DIR = (f'{PATH}/.Testing/')
#IMG_SIZE= (48, 48)
IMG_SIZE= (150, 150)

In [31]:
x_train, y_train, train_labels = load_data(TRAIN_DIR, IMG_SIZE)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:14<00:00,  2.49s/it]

603 images loaded from /home/alan/Documents/source_code/python/data/.Training/ directory.


In [123]:

#face_recognizer = cv2.face.EigenFaceRecognizer_create()
#face_recognizer = cv2.face.FisherFaceRecognizer_create()
#face_recognizer = cv2.face.LBPHFaceRecognizer_create()
# Leyendo el modelo
#face_recognizer.read('modeloEigenFace.xml')
#face_recognizer.read('modeloFisherFace.xml')
face_recognizer.read('modeloLBPHFace.xml')

In [124]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.read('modeloLBPHFace.xml')

In [52]:
from multiprocessing  import Pool
def f(i):
    if "modeloEigenFace.xml"== i[0]:
        face_recognizer = cv2.face.EigenFaceRecognizer_create()
    if "modeloFisherFace.xml"== i[0]:
        face_recognizer = cv2.face.FisherFaceRecognizer_create()
    if "modeloLBPHFace.xml"== i[0]:
        face_recognizer = cv2.face.LBPHFaceRecognizer_create()
    face_recognizer.read(i[0])
    return face_recognizer.predict(i[0])
resultados = []
for model,ruta in zip([modelos[2]],[rutas_modelos[2]]):
    resultado = []
    with Pool(32) as p:
        result = p.map_async(f, [(ruta,i) for i in x_train])

        for rs in result.get():
            resultado.append(rs)
        result.wait()
    resultados.append(resultado)

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'predict'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [ ]:
resultados

In [55]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.read('modeloLBPHFace.xml')

In [56]:
entrenamiento_evaluado = [ face_recognizer.predict(i)[1] for i in x_train]

In [57]:
entrenamiento_evaluado

[94.76469820619405,
 104.79188266467744,
 94.38827789793021,
 105.26484359922381,
 89.15662315110512,
 93.78465099610418,
 101.35611664616555,
 93.78465099610418,
 99.73053793149538,
 116.46354102259993,
 94.84143530152768,
 78.84401640229987,
 87.14963243146335,
 94.37190471706113,
 78.62152706195714,
 91.98626640623519,
 91.38831775210701,
 95.65656592691165,
 103.75713677107748,
 99.9161104372134,
 87.77127533234973,
 109.50593000158756,
 90.83021884199941,
 89.96884328876513,
 90.83233395061103,
 104.37682210613781,
 92.20569353764505,
 88.30847633516059,
 78.211576154341,
 91.28027088394242,
 96.68417119955019,
 105.99938227718897,
 82.73517677057333,
 84.86022343535258,
 92.01180113988923,
 98.17720053514165,
 81.185591110826,
 83.97785698379377,
 100.43703680208694,
 87.65364027936768,
 83.9752133638951,
 103.89527852952509,
 90.99876924906872,
 88.88155595149853,
 93.05882134655552,
 102.1464273493501,
 92.8770833426891,
 86.58915404263519,
 105.19448946412618,
 94.587937323144

In [58]:
face_recognizer = cv2.face.EigenFaceRecognizer_create()
face_recognizer.read('modeloEigenFace.xml')

In [59]:
entrenamiento_evaluado1 = [ face_recognizer.predict(i)[1] for i in x_train]
entrenamiento_evaluado1

[4074.841023365989,
 7469.90214754689,
 10983.698698411186,
 6141.924212842566,
 6781.909662276599,
 8066.9355622411,
 9752.51254685043,
 8066.9355622411,
 8024.542439118213,
 6906.253432516719,
 7608.447421351312,
 6066.1163345559835,
 6092.317642028278,
 8142.316391068718,
 4827.095458241027,
 7720.388780273032,
 5580.602649246366,
 7167.99042290433,
 9180.901730532243,
 10289.258091146818,
 6446.193169992867,
 7467.850610994829,
 8051.275994403756,
 4863.9825216198215,
 6233.1561788033,
 6744.862144885016,
 9016.37782207279,
 8585.962730382118,
 6123.63422050103,
 7552.2913841230065,
 3795.074229655809,
 10456.507246063047,
 6354.157152320205,
 8340.940956771117,
 8984.02560143746,
 5764.605368902758,
 7121.156806318871,
 7526.333717304951,
 8271.465900270958,
 4839.632547806523,
 10820.815507691514,
 9273.780101774191,
 5762.920316447355,
 5187.97196575497,
 9951.41066620192,
 8722.078711979006,
 7563.676156045613,
 8637.335791881495,
 7017.816341014701,
 9698.609338465969,
 10314.

In [60]:
face_recognizer = cv2.face.FisherFaceRecognizer_create()
face_recognizer.read('modeloFisherFace.xml')
entrenamiento_evaluado2 = [ face_recognizer.predict(i)[1] for i in x_train]
entrenamiento_evaluado2

[41.63552416307835,
 40.30882296125848,
 33.01057585613928,
 27.0056265439794,
 19.300557341583218,
 63.80137242280115,
 15.081265145752113,
 63.80137242280115,
 12.410220300180782,
 12.805917028828102,
 15.414069237561792,
 23.703481811387377,
 21.93270703539256,
 8.93986319733865,
 40.83998486511407,
 23.17096108172494,
 23.678990841926684,
 13.163338973045455,
 36.994498153643256,
 7.201228150549837,
 4.493336633699793,
 15.273783087691422,
 22.749509986793004,
 7.291624150687516,
 46.21002595596592,
 1.5692413395233396,
 79.08327842111835,
 23.68503682435904,
 4.933158354919845,
 36.53076271967668,
 8.718945560741739,
 57.696180572291205,
 0.17377329561385113,
 33.51423318996391,
 29.351423514585704,
 0.2264842853399458,
 13.142892310559413,
 20.511065576199954,
 12.813798158982086,
 0.22364001333440475,
 9.094076436962627,
 18.91515644225864,
 68.06142076290571,
 0.10686988837737688,
 36.76709656483787,
 62.81609718316407,
 3.5020637155292795,
 32.0175099936981,
 12.27886762364839

In [64]:
len([i for i in entrenamiento_evaluado1 if not i < 5700])

529

In [68]:
len([i for i in entrenamiento_evaluado2 if  i < 500])

603

In [67]:
len([i for i in entrenamiento_evaluado if i < 70])

0

In [25]:
len(x_train)

NameError: name 'x_train' is not defined

In [18]:
imagePath = "/home/alan/Documents/source_code/python/data/"
cascPath = "/home/alan/Documents/source_code/python/data/casc"

In [19]:
image = cv2.imread(imagePath)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

error: OpenCV(4.6.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [17]:
faceCascade = cv2.CascadeClassifier(cascPath)
faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.cv.CV_HAAR_SCALE_IMAGE
)
for (x, y, w, h) in faces:
     cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)


NameError: name 'cascPath' is not defined

In [12]:
model = cv2.face.LBPHFaceRecognizer_create()

In [13]:
model.predict()

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'predict'
> Overload resolution failed:
>  - face_FaceRecognizer.predict() missing required argument 'src' (pos 1)
>  - face_FaceRecognizer.predict() missing required argument 'src' (pos 1)
